In [87]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from sklearn.preprocessing import LabelEncoder
import faiss
import PIL
from PIL import Image


In [88]:
data_dir = './images'

In [89]:
def load_images(data_dir):
    images = []
    labels = []
    
    # Iterate over each folder (representing Pokémon species)
    for folder in os.listdir(data_dir):
        folder_path = os.path.join(data_dir, folder)
        
        # Check if it's a directory (folder)
        if os.path.isdir(folder_path):
            # Iterate over each image file in the folder
            img_path = os.path.join(folder_path, '0.jpg')
            img = image.load_img(img_path, target_size=(224, 224))
            img_array = image.img_to_array(img)
            img_array = preprocess_input(img_array)
            images.append(img_array)
            labels.append(folder)

    # Convert lists to numpy arrays
    return np.array(images), np.array(labels)

In [90]:
images, labels = load_images(data_dir)
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)

In [91]:
print(images.shape)
print(labels.shape)

(898, 224, 224, 3)
(898,)


In [93]:
base_model = VGG16(weights='imagenet', include_top=False, pooling='avg')

# Extract features from images
features = base_model.predict(images)

# Save the features and labels
np.save('pokemon_features.npy', features)
np.save('pokemon_labels.npy', labels_encoded)

29/29 ━━━━━━━━━━━━━━━━━━━━ 62s 2s/step


In [94]:
# Load features and set up FAISS index
features = np.load('pokemon_features.npy')
labels_encoded = np.load('pokemon_labels.npy')

In [95]:
index = faiss.IndexFlatL2(features.shape[1])
index.add(features)
faiss.write_index(index, 'pokemon_index.index')

In [96]:
def find_closest_pokemon(img_path, model, index, labels):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)
    img_features = model.predict(img_array)
    
    distances, indices = index.search(img_features, 1)
    closest_pokemon_idx = indices[0][0]
    closest_pokemon_label = labels[closest_pokemon_idx]
    
    return closest_pokemon_label

In [120]:
input_image_path = 'images/Totodile/1.jpg'  # Replace with the actual path to your test image
closest_pokemon = find_closest_pokemon(input_image_path, base_model, index, labels_encoded)
print(f"The closest Pokémon is: {le.inverse_transform([closest_pokemon])[0]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
The closest Pokémon is: Electrike


In [ ]:
print(labels_encoded)